<a href="https://colab.research.google.com/github/ShenggKai/Kaggle_TempoRun/blob/master/Notebook/baseliene.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

link colab: https://colab.research.google.com/drive/1W2FUkT9pLFhHBbuorcmRSII_5z9r3xAu?usp=sharing

# Download data

In [ ]:
import json
# lưu ý trên Kaggle có sẵn data nên mình chỉ cần dẫn link vào
# #Download lyrics
# !gdown --id 1V7WxGaVv3UamD4AM5qnqDNzkBSw6paQy
# #Download queries
# !gdown --id 1srNAjKMTncZUgxDTZZHfMnTbPd9ic2xx

# Preliminaries & Corpus Creation

In [ ]:
# Install required libraries
!pip install rank-bm25

In [ ]:
!pip install -q tf-models-official==2.3.0

     |████████████████████████████████| 840 kB 2.7 MB/s 
     |████████████████████████████████| 352 kB 61.4 MB/s 
     |████████████████████████████████| 234 kB 53.4 MB/s 
     |████████████████████████████████| 99 kB 9.1 MB/s 
     |████████████████████████████████| 47.8 MB 2.4 MB/s 
     |████████████████████████████████| 1.2 MB 44.9 MB/s 
     |████████████████████████████████| 1.1 MB 14.9 MB/s 
     |████████████████████████████████| 462 kB 49.6 MB/s 


In [ ]:
# Import required libraries
import os
import pprint as pp
import numpy as np
import json
import tensorflow as tf 
from gensim.corpora import WikiCorpus
from rank_bm25 import BM25Okapi, BM25Plus

from official.modeling import tf_utils
from official import nlp
from official.nlp import bert

# Load the required submodules
import official.nlp.optimization
import official.nlp.bert.bert_models
import official.nlp.bert.configs
import official.nlp.bert.run_classifier
import official.nlp.bert.tokenization
import official.nlp.data.classifier_data_lib
import official.nlp.modeling.losses
import official.nlp.modeling.models
import official.nlp.modeling.networks

In [ ]:
from tqdm import tqdm
# link to data
CORPUS_SONG_JSON = "../input/temporun-retrieval-of-song-lyrics/2335_Songs.json"
QUERIES_JSON = "../input/temporun-retrieval-of-song-lyrics/500_queries.json"

In [ ]:
with open(CORPUS_SONG_JSON, 'r', encoding='utf-8') as fo:
  corpus_data = json.load(fo)

with open(QUERIES_JSON, 'r', encoding='utf-8') as fo:
  queries_test = json.load(fo)

FileNotFoundError: ignored

In [ ]:
def read_corpus(corpus_data):
    """
    Function that iteratively reads the saved articles from the corpus directory
    and appends the text to a list.
    
    Parameters
    ----------
    @param corpus_directory: str,
        A valid string specifying the path to the local directory in which the 
        files were saved using make_corpus().
        
    Returns
    -------
    @return corpus, list
        A list containing the text of an article in each element.
    """
    
    # Initialize empty corpus list
    corpus = []
    
    # For each file in the corpus directory, do...
    print("Reading local corpus, please wait...")

    for song in corpus_data:
      if song["lyrics"].split("\n")[0].startswith("Bài hát: "):
          lyric = "\n".join(song["lyrics"].split("\n")[1:])
      else:
          lyric = song["lyrics"]
      corpus.append([song["song_id"], lyric])
        
    # Done, return
    print("Done!")
    return corpus

In [ ]:
# Read corpus! 
corpus = read_corpus(corpus_data)

# Look at some example...
corpus[2]

# For some reason (only in Google Colab), this cell might need to be stopped
# for the first run, then run again. 

Reading local corpus, please wait...
Done!


['0002',
 'Đoạn buồn cho tôi biết mấy\nNgười tình rời xa tôi khuất bóng\nBiết cô đơn dâng ngập nỗi sầu \nRồi từng chiều mấy đen tim tím\nMột mình buồn tôi bước đơn côi \nMang trong tim nỗi nhớ một mình\nCho trời chiều buồn mênh mang\nLang thang mình tôi bước trên nỗi sầu\nĐi đâu về đâu muôn lối vô định\nĐoạn buồn biết mấy trong tôi\nNhớ người không nhớ đến tôi\nChiều tím tôi gọi tên ai \nĐoạn buồn cho tôi biết mấy\nHẹn hò em đã ngó lơ\nChiều nay tôi đứng một mình\nDòng người đua chen khắp chốn\nNhìn hoài không thấy em sang\nCho tôi đoạn buồn tình nhân.\n \n']

# BM25 Implementation

In [ ]:
!pip install underthesea
!pip install pyvi

     |████████████████████████████████| 7.6 MB 555 kB/s            
     |████████████████████████████████| 581 kB 44.2 MB/s            
     |████████████████████████████████| 743 kB 42.1 MB/s            
     |████████████████████████████████| 8.5 MB 549 kB/s            


In [ ]:
# Tokenize the corpus
from underthesea import word_tokenize

tokenized_corpus = []
for doc in tqdm(corpus):
  tokenize_lyric = word_tokenize(doc[-1]) 
  tokenize_lyric.append(f"-->{doc[0]}")
  tokenized_corpus.append(tokenize_lyric)

100%|██████████| 2335/2335 [00:43<00:00, 54.24it/s]


In [ ]:
tokenized_corpus[1]

['Từ',
 'ngày',
 'con',
 'ra',
 'đi',
 'Cha',
 'lòng',
 'buồn',
 'thương nhớ',
 'Cô đơn',
 'và',
 'sầu muộn',
 'Bao',
 'ngày tháng',
 'mong chờ',
 'Từ',
 'ngày',
 'con',
 'ra',
 'đi',
 'Tới',
 'thời gian',
 'bây giờ',
 'Thấm thoát',
 'mười',
 'hai',
 'năm',
 'chẵn',
 'Cha',
 'thường',
 'đứng',
 'ngồi',
 'ngẩn ngơ',
 '[',
 'ĐK',
 ':',
 ']',
 'Mẹ con',
 'không',
 'còn',
 'nữa',
 'Đi',
 'rồi',
 'quên',
 'từ',
 'giã',
 'Cha',
 'một mình',
 'thui thủi',
 'Lạnh giá',
 'và',
 'đơn côi',
 'Nay',
 'con',
 'ở',
 'phương',
 'xa',
 'Đừng',
 'quên',
 'nơi',
 'quê',
 'nhà',
 'Vẫn',
 'còn',
 'người',
 'trông đợi',
 'Người',
 'ấy',
 'là',
 'cha',
 'già',
 'Cha',
 'chờ',
 'con',
 'trong',
 'hấp hối',
 'Lá',
 'thư',
 'cuối cùng',
 'con',
 'ơi',
 'Thư',
 'này',
 'mong',
 'con',
 'nhận',
 'được',
 'Thương',
 'con',
 'vất vả',
 'đơn côi',
 'Mưa',
 'lạnh',
 'ngoài trời',
 'đang',
 'rơi',
 'Gắng',
 'thêm',
 'một chút',
 'tàn hơi',
 'Cha',
 'đây',
 'nhắn',
 'lại',
 'một',
 'lời Chúc',
 'con',
 'hạnh phúc',
 '

In [ ]:
# Instantiate BM25 object from the tokenized corpus
bm25 = BM25Okapi(tokenized_corpus)

In [ ]:
#hàm a nhận một truy vấn chuỗi và đánh số `n` kết quả bắt buộc, 
#và trả về kết quả có liên quan nhất` n` từ kho dữ liệu của chúng ta:
def bm25okapi_search(tokenized_query, bm25, corpus, n_results = 1):
    """
    Function that takes a tokenized query and prints the first 100 words of the 
    n_results most relevant results found in the corpus, based on the BM25
    method.
    
    Parameters
    ----------
    @param tokenized_query: list, array-like
        A valid list containing the tokenized query.
    @param bm25: BM25 object,
        A valid object of type BM25 (BM25Okapi or BM25Plus) from the library
        `rank-bm25`, initialized with a valid corpus.
    @param corpus: list, array-like
        A valid list containing the corpus from which the BM25 object has been 
        initialized. As returned from function read_corpus().
    @param n_results: int, default = 1
        The number of top results to print.
    """
    
    # We skip checking validity of arguments for now... We assume the user 
    # knows what they're doing.
    
    # Get top results for the query
    top_results = bm25.get_top_n(tokenized_query, corpus, n = n_results)
    
    # Take words from each result
    top_results_id = [' '.join(top_result).split("-->")[-1] 
                             for top_result in top_results]
    
    # Return results
    return top_results_id

In [ ]:
# check queries
queries_test[0]

{'query_id': '000', 'query': 'cho qua bao năm mộng buon quen dấu'}

# Process & Export file submission

In [ ]:
import csv
from pyvi import ViUtils
import re
# trong phần output
with open("submission.csv", "w") as csv_writer:
  fieldnames = ['query_id', 'song_id']
  writer = csv.DictWriter(csv_writer, fieldnames=fieldnames)
  writer.writeheader()
  for ques in tqdm(queries_test):
    preprocess_query = " ".join(ques["query"].lower().replace("/", "").strip("").split())
    tokenized_query = word_tokenize(preprocess_query)
    results = bm25okapi_search(tokenized_query = tokenized_query, bm25 = bm25, corpus = tokenized_corpus, n_results = 10)
    writer.writerow({'query_id': ques["query_id"], "song_id": " ".join(results)})

100%|██████████| 500/500 [00:04<00:00, 106.41it/s]
